In [1]:
# import packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import time 
import os
import json
import urllib


In [2]:
# Some urls are blocked from untrusted sites, the below code make python environment to trust every site
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

In [5]:
def download_images():
    
    f = open('site_urls.txt','r')
    search_url = f.readlines()[5]
    print(search_url)
   
    # path of chorme drive to run selenium
    path = r'chromedriver'
    
    driver = webdriver.Chrome(path)
    
    k = open('wildlife keywords.txt','r')
    kw = k.readlines()
    
    for i in range(0,len(kw)):
        print(i)
        keyW = kw[i]
        
        driver.get(search_url)
        time.sleep(5)
        
        # X mark clicking in search text box
        driver.find_element_by_xpath('/html/body/tm-root/div[1]/main/div/ng-component/tm-search-header/div[1]/div/tm-refine-header/div[1]/tm-refine-keywords/div/form/tg-input-container/label[2]/span[3]/button').click()
        time.sleep(2)
        
        # search text box 
        box = driver.find_element_by_xpath('/html/body/tm-root/div[1]/main/div/ng-component/tm-search-header/div[1]/div/tm-refine-header/div[1]/tm-refine-keywords/div/form/tg-input-container/label[2]/span[2]/input')    
        #sending the keyword
        box.send_keys(keyW)
        time.sleep(10)
        
        # try block if there are no ad results for the keyword it will continue with next keyword
        try:
            # collecting links of ad results
            links = driver.find_element_by_xpath('/html/body/tm-root/div[1]/main/div/ng-component/div/div/tg-row/tg-col/tm-search-results/div/div[2]/tm-potential-r18-content/div/tg-row')
            sub_lnks = links.find_elements_by_tag_name('a')

            lnks_lst = []

            for lnk in sub_lnks:
                lnks_lst.append(lnk.get_attribute('href'))
            
             # making a directory like URL+Keyword_number , example 5_0 , 5_1,...
            if lnks_lst:
                if not os.path.exists(str(5)+'_'+str(i)):
                    os.mkdir(str(5)+'_'+str(i))
                    
            #  iterating through ads results 
            for j in range(1,3):
                driver.get(lnks_lst[j])
                time.sleep(10)
                # Title capturing
                sub_txt = driver.find_element_by_xpath('/html/body/tm-root/div[1]/main/div/tm-marketplace-listing/div/div[1]/tm-potential-r18-content/div/tg-row[1]/tg-col[2]/tm-marketplace-buyer-options/h1').text
                #making directory inside the above created directory like 0 means first ad, 1 for second ad....
                if not os.path.exists(str(5)+'_'+str(i) +'/'+ str(j)):
                    os.mkdir(str(5)+'_'+str(i) +'/'+ str(j))
                # making dictionary
                details = {
                    'name' : sub_txt,
                    'price' : driver.find_element_by_xpath('/html/body/tm-root/div[1]/main/div/tm-marketplace-listing/div/div[1]/tm-potential-r18-content/div/tg-row[1]/tg-col[2]/tm-marketplace-buyer-options/div/tm-buy-now-box/tg-box2/div/div[1]/p[2]/strong').text,
                    'location': driver.find_element_by_xpath('//*[@id="seller-details-section"]/tm-marketplace-listing-seller-details/tg-row/tg-col[2]/tm-member-summary/section/div[3]/tm-core-seller-details/tg-rack/tg-rack-item[1]/div/div/tg-rack-item-secondary').text,
                    'details': driver.find_element_by_xpath('/html/body/tm-root/div[1]/main/div/tm-marketplace-listing/div/div[1]/tm-potential-r18-content/div/tg-row[1]/tg-col[1]/tm-marketplace-listing-body/div/div[2]').get_attribute("innerText"),
                    'description' : driver.find_element_by_xpath('/html/body/tm-root/div[1]/main/div/tm-marketplace-listing/div/div[1]/tm-potential-r18-content/div/tg-row[1]/tg-col[1]/tm-marketplace-listing-body/div/div[1]').get_attribute("innerText"),
                    'url' : lnks_lst[j]
                }
                # writing the dictionary to text file like 0.txt for first ad ....
                with open(str(5)+'_'+str(i) +'/'+ str(j) + '/' + str(j) + '.txt', 'w') as tfile:
                     tfile.write(json.dumps(details))
                # collecting image urls
                image_lst = []
                elem1 = driver.find_element_by_xpath('/html/body/tm-root/div[1]/main/div/tm-marketplace-listing/div/div[1]/tm-potential-r18-content/div/tg-row[1]/tg-col[1]/tm-marketplace-listing-photos/tg-content-slider2/div/div[2]/div/div/div')
                sub = elem1.find_elements_by_tag_name('tg-aspect-ratio')
                for k,l in enumerate(sub):

                        urls = l.get_attribute('style');
                        src = urls.split('"')[1].replace('thumb','full')
                        print(src)
                        image_lst.append(src)
                        
                        # collecting images and saving it inside the above created directory like 0_0.jpg for first ad first image
                        # 0_1.jpg for first ad second image ...
                        if src != None:
                            urllib.request.urlretrieve(str(src), os.path.join(str(5)+'_'+str(i), str(j), str(j)+'_'+str(k)+'.jpg'))
                # writing image urls to text file like 0_imgs.txt for first ad ....
                with open(str(5)+'_'+str(i) +'/'+ str(j) + '/' + str(j) + '_imgs.txt', 'w') as tfile:
                     tfile.write(json.dumps(image_lst))
        except:
            continue

    driver.close()
    

In [ ]:
def main():
    download_images()
    
if __name__ == "__main__":
    main()